In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# load the cleaned dataset
data_path = "cleaned_diabetic_data.csv"
df = pd.read_csv(data_path)
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,readmitted,readmitted_binary,admission_type,discharge_disposition,admission_source,diag_1_group,diag_2_group,diag_3_group,med_count,age_numeric
0,12522,48330783,Caucasian,Female,[80-90),2,1,4,13,68,...,NO,0,Urgent,Discharged to home,Transfer from hospital,Circulatory,Circulatory,Infectious and parasitic diseases,2,85
1,15738,63555939,Caucasian,Female,[90-100),3,3,4,12,33,...,NO,0,Elective,To SNF,Transfer from hospital,Circulatory,Neoplasms,Respiratory,2,95
2,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,51,...,NO,0,Emergency,Discharged to home,Emergency Room,Neoplasms,Neoplasms,Diabetes,2,45
3,28236,89869032,AfricanAmerican,Female,[40-50),1,1,7,9,47,...,>30,0,Emergency,Discharged to home,Emergency Room,Diabetes,Circulatory,Injury,1,45
4,35754,82637451,Caucasian,Male,[50-60),2,1,2,3,31,...,>30,0,Urgent,Discharged to home,Clinic Referral,Circulatory,Circulatory,Diabetes,1,55


In [ ]:
# Lets build a xgboost model to predict readmission
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Define features and target variable

X = df.drop(columns=['readmitted_binary', 'diag_1', 'diag_2', 'diag_3', 'age', 'encounter_id', 'patient_nbr', 'readmitted'])
y = df['readmitted_binary']

X_encoded = pd.get_dummies(X, drop_first=True)

# Split the dataset into training and testing and validation sets
X_train, X_temp, y_train, y_temp = train_test_split(X_encoded, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

def train_xgboost_model(X_train, y_train, random_seed=42, scale_pos_weight=None):
    xgb_model = xgb.XGBClassifier(
        random_state=random_seed,
        eval_metric="logloss",
        enable_categorical=True,
        n_estimators=100,
        scale_pos_weight=scale_pos_weight
    )

    # Fit the model
    xgb_model.fit(X_train, y_train)
    return xgb_model

# Train the model
xgb_model = train_xgboost_model(X_train, y_train)
# Evaluate the model on the validation set
from sklearn.metrics import accuracy_score, classification_report
y_pred_val = xgb_model.predict(X_val)
accuracy_val = accuracy_score(y_val, y_pred_val)
print(f"Validation Accuracy: {accuracy_val:.4f}")
print(classification_report(y_val, y_pred_val))
# Evaluate the model on the test set
y_pred_test = xgb_model.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred_test)
print(f"Test Accuracy: {accuracy_test:.4f}")
print(classification_report(y_test, y_pred_test))


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:race: object, gender: object, max_glu_serum: object, A1Cresult: object, metformin: object, repaglinide: object, nateglinide: object, chlorpropamide: object, glimepiride: object, acetohexamide: object, glipizide: object, glyburide: object, tolbutamide: object, pioglitazone: object, rosiglitazone: object, acarbose: object, miglitol: object, troglitazone: object, tolazamide: object, examide: object, citoglipton: object, insulin: object, glyburide-metformin: object, glipizide-metformin: object, glimepiride-pioglitazone: object, metformin-rosiglitazone: object, metformin-pioglitazone: object, change: object, diabetesMed: object, admission_type: object, discharge_disposition: object, admission_source: object, diag_1_group: object, diag_2_group: object, diag_3_group: object